# Reference
- [transfer-1](https://officeguide.cc/pytorch-transfer-learning-resnet18-classify-mnist-tutorial-examples/)
- [transfer-2](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pydicom
import copy
import time

import torch
from torch import nn, optim
from torchvision import transforms, io, models
from torchvision.transforms import functional as F

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
import random
np.random.seed(2022)
random.seed(2022)
torch.manual_seed(2022)

In [3]:
def display(tr: torch.Tensor):
    infos = {
        'min': torch.amin(tr),
        'max': torch.amax(tr),
        'dtype': tr.dtype,
        'size': tr.size()
    }

    return infos

In [4]:
class SPECTDataset(torch.utils.data.Dataset):
    '''
    - split data into train, val (frac, 1-frac)
    - random_state set 2022 (fix random result)
    '''
    def __init__(self, root, train, frac, transform):
        self.root = Path(root)
        self.transform = transform
        df = pd.read_csv( str(self.root/ "DICOM/train.csv") )

        # Train / Validation data
        train_df = df.sample(frac=frac, random_state=2022, ignore_index=True)
        if train: self.list = train_df
        else: self.list = pd.concat( [df, train_df] ).drop_duplicates(keep=False, ignore_index=True)

        # edit file path
        self.list.FilePath = self.list.FilePath.apply(lambda _: self.root / _[1:])

    def __len__(self):
        return len(self.list)

    def __getitem__(self, idx):
        dcm = pydicom.read_file( str(self.list.FilePath[idx]) )

        # age, gender
        age = self.list.loc[idx, 'Age']
        gender = self.list.loc[idx, 'Gender']
        
        # label (1,2,3 -> 0.,1,2)
        label = int(self.list.Stage[idx]) - 1

        # Preprocessed Pixels: totensor, 3 channel
        pixel = dcm.pixel_array[ self.list.loc[idx, 'index'] ] # 用 index 當 column name 真的是天才
        # low, high = self.get_low_high(dcm)
        # pixeled = self.getWindow(pixel, low, high)
        # img = (pixeled - np.min(pixeled)) / (np.max(pixeled) - np.min(pixeled))
        img = torch.tensor(pixel.astype(np.float32))
        img = torch.stack([img, img, img], dim=0)

        seed = np.random.randint(1e9)
        random.seed(seed)
        torch.manual_seed(seed)

        img = self.transform(img)

        return img, age, gender, label


In [5]:
batch_size = 8
split = .8
shuffle_dataset = True
random_seed= 2022
num_epochs = 10

lr = 1e-4

In [6]:
preprocess = transforms.Compose([
    transforms.CenterCrop(50), 
    # transforms.Normalize((62.2852, 62.2852, 62.2852), (76.8448, 76.8448, 76.8448)), # 跑 normalize 反而下降準確度
    transforms.Resize(224),
])

In [7]:
training_data = SPECTDataset(root="./data", train=True, frac=split, transform=preprocess)
validation_data = SPECTDataset(root="./data", train=False, frac=split, transform=preprocess)

In [8]:
print("訓練資料集數量：", len(training_data))
print("測試資料集數量：", len(validation_data))

訓練資料集數量： 129
測試資料集數量： 32


In [9]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size)

In [10]:
for X, age, gender, y in val_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    print("Age: ", age)
    print("Gender: ", gender)

    break

Shape of X [N, C, H, W]:  torch.Size([8, 3, 224, 224])
Shape of y:  torch.Size([8]) torch.int64
Age:  tensor([54, 77, 59, 54, 75, 48, 25, 73])
Gender:  tensor([1, 1, 0, 0, 1, 1, 1, 0])


In [11]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

Using cache found in /home/azetry/.cache/torch/hub/pytorch_vision_v0.10.0
/home/azetry/.conda/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/azetry/.conda/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
class SPECT_ResNet50(nn.Module):
    def __init__(self):
        super(SPECT_ResNet50, self).__init__()

        # 載入 ResNet50 類神經網路結構
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

        # 鎖定 ResNet50 預訓練模型參數
        for param in self.model.parameters():
           param.requires_grad = False

        # 修改輸出層輸出數量
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 200),
            nn.Linear(200, 20)
        )

    def forward(self, x, age, gender):
        logits_ = self.model(x)

        # Add Age and Gender
        age.unsqueeze_(1)
        logits_ = torch.cat((logits_, age), dim=1)

        gender.unsqueeze_(1)
        logits_ = torch.cat((logits_, gender), dim=1)

        # Final Classifier (這樣寫不太好，但就先可以run)
        logits = nn.Linear(22, 3).to(device)(logits_)

        return logits

In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda:0 device


In [15]:
model = SPECT_ResNet50().to(device)
print(model)

Using cache found in /home/azetry/.cache/torch/hub/pytorch_vision_v0.10.0


SPECT_ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [16]:
# 損失函數
loss_fn = nn.CrossEntropyLoss()

# 學習優化器
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    # 資料總筆數
    size = len(dataloader.dataset)

    # 將模型設定為訓練模式
    model.train()

    # 批次讀取資料進行訓練
    for batch, (X, age, gender, y) in enumerate(dataloader):
        # 將資料放置於 GPU 或 CPU
        X, age, gender, y = X.to(device), age.to(device), gender.to(device), y.to(device)

        pred = model(X, age, gender)         # 計算預測值
        loss = loss_fn(pred, y) # 計算損失值（loss）

        optimizer.zero_grad()   # 重設參數梯度（gradient）
        loss.backward()         # 反向傳播（backpropagation）
        optimizer.step()        # 更新參數

        # 輸出訓練過程資訊
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
# 測試模型
def test(dataloader, model, loss_fn):
    # 資料總筆數
    size = len(dataloader.dataset)

    # 批次數量
    num_batches = len(dataloader)

    # 將模型設定為驗證模式
    model.eval()

    # 初始化數值
    test_loss, correct = 0, 0

    # 驗證模型準確度
    with torch.no_grad():  # 不要計算參數梯度
        for X, age, gender, y in dataloader:
            # 將資料放置於 GPU 或 CPU
            X, age, gender, y = X.to(device), age.to(device), gender.to(device), y.to(device)

            # 計算預測值
            pred = model(X, age, gender)

            # 計算損失值的加總值
            test_loss += loss_fn(pred, y).item()

            # 計算預測正確數量的加總值
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # 計算平均損失值與正確率
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
# 開始訓練模型
for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(val_loader, model, loss_fn)
print("完成！")

Epoch 1
-------------------------------
loss: 5.126586  [    0/  129]
Test Error: 
 Accuracy: 3.1%, Avg loss: 7.092352 

Epoch 2
-------------------------------
loss: 9.946861  [    0/  129]
Test Error: 
 Accuracy: 28.1%, Avg loss: 8.210431 

Epoch 3
-------------------------------
loss: 14.613620  [    0/  129]
Test Error: 
 Accuracy: 3.1%, Avg loss: 11.691108 

Epoch 4
-------------------------------
loss: 13.190536  [    0/  129]
Test Error: 
 Accuracy: 28.1%, Avg loss: 6.489862 

Epoch 5
-------------------------------
loss: 4.756152  [    0/  129]
Test Error: 
 Accuracy: 68.8%, Avg loss: 4.806487 

Epoch 6
-------------------------------
loss: 4.703821  [    0/  129]
Test Error: 
 Accuracy: 43.8%, Avg loss: 8.049026 

Epoch 7
-------------------------------
loss: 14.396820  [    0/  129]
Test Error: 
 Accuracy: 3.1%, Avg loss: 9.346836 

Epoch 8
-------------------------------
loss: 3.420487  [    0/  129]
Test Error: 
 Accuracy: 43.8%, Avg loss: 6.000332 

Epoch 9
---------------

In [20]:
for i in range(2):
    b = torch.randn(3)
    b.unsqueeze_(0)
    if i == 0:
        a = b
    else:
        a = torch.cat((a, b), dim=0)

In [21]:
a

tensor([[0.4607, 0.9535, 1.9397],
        [0.1768, 2.4619, 0.5424]])

In [22]:
b = torch.randn(2)

In [23]:
b.unsqueeze_(1)

tensor([[-0.6961],
        [ 0.3229]])

In [24]:
torch.cat((a, b), dim=1)

tensor([[ 0.4607,  0.9535,  1.9397, -0.6961],
        [ 0.1768,  2.4619,  0.5424,  0.3229]])